In [49]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

web_driver = webdriver.Chrome()

In [ ]:
ress_parts = []
ress_models = []
ress_grades = []


# 車種選択画面

web_driver.get("https://toyota.jp/service/estimate/options")
time.sleep(5)
elms_cars = web_driver.find_elements(By.CLASS_NAME, "myt-cars-module")

for model_id in range(len(elms_cars)):
    if model_id>0:
        web_driver.get("https://toyota.jp/service/estimate/options")
        time.sleep(5)
        elms_cars = web_driver.find_elements(By.CLASS_NAME, "myt-cars-module")
    car_elm = elms_cars[model_id]
    car_data = car_elm.text
    car_data_list = car_data.split("\n")
    res_car = {
        "model_id": model_id,
        "model_name": car_data_list[0],
        "price_low": car_data_list[1].replace("円（税込）", "").replace("～", "").replace(",", ""),
        "price_high": (
            car_data_list[2].replace("円（税込）", "").replace("～", "").replace(",", "") if len(car_data_list)==3 
            else car_data_list[1].replace("円（税込）", "").replace("～", "").replace(",", "")
        ),
        "img_url": car_elm.find_element(By.TAG_NAME, "img").get_attribute(name="src")
    }

    # 対象車種ページに移動
    car_elm.click()
    ress_models.append(res_car)
    time.sleep(5)

    # グレード選択
    elms = web_driver.find_elements(By.CLASS_NAME, "myt-register-header-carname-area-subunit")
    elms[0].click()#グレードを変更する
    time.sleep(5)
    grade_elms = web_driver.find_elements(By.CLASS_NAME, "gtmModal.myt-grade-module")#グレードの取得
    # for grade_id in range(len(grade_elms)):
    for grade_id in range(len(grade_elms)):
        if grade_id>0:
                elms = web_driver.find_elements(By.CLASS_NAME, "myt-register-header-carname-area-subunit")
                elms[0].click()#グレードを変更する
                time.sleep(3)
                grade_elms = web_driver.find_elements(By.CLASS_NAME, "gtmModal.myt-grade-module")#グレードの取得
        elm_grade = grade_elms[grade_id]
        grade_text_list = elm_grade.text.split("\n")
        res_grade = {
            "model_id": model_id,
            "grade_id": grade_id,
            "grade_name": grade_text_list[0],
            "price": grade_text_list[1].replace(",", "").replace(" ", "").replace("円", ""),
            "desc": grade_text_list[2],
        }
        ress_grades.append(res_grade)
        print(res_grade)
        elm_grade.click() #OK
        time.sleep(5)
        elms = web_driver.find_elements(By.CLASS_NAME, "myt-btn.myt-btn--red.myt-btn--half-pc")#対象グレードのページに遷移
        try:
            elms[0].click()
            time.sleep(5)
        except:
             pass
    
        # オプション選択
        elms = web_driver.find_elements(By.CLASS_NAME, "myt-estimate-sim-btn-inner")
        elms[1].click()#オプションを開く
        time.sleep(4)

        elms = web_driver.find_elements(By.CLASS_NAME, "myt-estimate-sim-option")
        elms_parts = elms[0].find_elements(By.CLASS_NAME, "myt-estimate-sim-option-unit-detail")
        elms_parts_img = elms[0].find_elements(By.CLASS_NAME, "myt-estimate-sim-option-unit-thumb")
        for option_id in range(len(elms_parts)):
            res_text = elms_parts[option_id].text.split("\n")
            res_option = {
                "option_id":option_id,
                "grade_id":grade_id,
                "model_id": model_id,
                "name": res_text[0],
                "type": res_text[1],
                "price": res_text[2].replace(",", "").replace("円", ""),
                "img_url": elms_parts_img[option_id].find_element(By.TAG_NAME, "img").get_attribute(name="src")
            }
            ress_parts.append(res_option)
        web_driver.find_elements(By.CLASS_NAME, "myt-content-header__back")[0].click()
        time.sleep(10)

In [52]:
pd.DataFrame(ress_parts).to_csv("../asset/exterior_parts.csv")

In [53]:
pd.DataFrame(ress_models).to_csv("../asset/models.csv")

In [ ]:
pd.DataFrame(ress_grades).to_csv("../asset/grades.csv")